In [80]:
from selenium import webdriver
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
import requests
from bs4 import BeautifulSoup
from html_table_parser import parser_functions
import pandas as pd

In [81]:
# 드라이버 초기화
driver = webdriver.Chrome()

url = "https://ai.dongguk.edu/page/30427"

### 지정된 url 에서 모든 테이블 값들 크롤링

In [82]:
selected_index = [0, 1, 3, 4, 5, 8, 10]
sequential_name = {0:"컴퓨터 심화", 1:"컴퓨터 일반", 2:"멀티미디어", 3:"인공지능 심화", 4:"인공지능 일반", 5:"데이터사이언스", 6:"엔터테인먼트"}

In [83]:
try:
    driver.get(url)
    element = WebDriverWait(driver, 10) # request wait 걸기
    html = driver.page_source # page_source 얻기
    soup = BeautifulSoup(html, 'html.parser') # get html
    tables = soup.select('.tbl') 

    # 테이블 데이터 추출
    def parse_table(table):
        # Tag 객체인지 확인
        if not table.name == 'table':
            return []
        
        rows = table.find_all('tr')
        data = []
        rowspan_map = {}  # rowspan을 처리하기 위한 맵

        for row_idx, row in enumerate(rows):
            row_data = []
            cells = row.find_all(['td', 'th'])
            col_idx = 0

            for cell in cells:
                while col_idx in rowspan_map:
                    # 이미 rowspan이 적용된 셀이 있는지 확인
                    row_data.append(rowspan_map[col_idx]['text'])
                    rowspan_map[col_idx]['count'] -= 1
                    if rowspan_map[col_idx]['count'] == 0:
                        del rowspan_map[col_idx]
                    col_idx += 1

                cell_text = cell.get_text(strip=True)
                rowspan = int(cell.get('rowspan', 1))
                colspan = int(cell.get('colspan', 1))

                for i in range(colspan):
                    row_data.append(cell_text)

                if rowspan > 1:
                    for i in range(colspan):
                        span_col_idx = col_idx + i
                        if span_col_idx not in rowspan_map:
                            rowspan_map[span_col_idx] = {'text': cell_text, 'count': rowspan - 1}
                        else:
                            rowspan_map[span_col_idx]['count'] += (rowspan - 1)

                col_idx += colspan

            while col_idx in rowspan_map:
                row_data.append(rowspan_map[col_idx]['text'])
                rowspan_map[col_idx]['count'] -= 1
                if rowspan_map[col_idx]['count'] == 0:
                    del rowspan_map[col_idx]
                col_idx += 1

            data.append(row_data)

        # 데이터 평탄화
        max_length = max(len(row) for row in data)
        flattened_data = [row + [''] * (max_length - len(row)) for row in data]

        return flattened_data

    all_table_data = []
    for index, table in enumerate(tables): 
        if index in selected_index:
            table_data = parse_table(table)
            find_index = selected_index.index(index)
            table_name = sequential_name.get(find_index, f"Table_{index + 1}")
            df = pd.DataFrame(table_data)
            all_table_data.append((table_name, df))

    # 출력 [0, 1, 3, 4, 5, 8, 10]
    for name, df in all_table_data:
        print(f"{name}:")
        display(df)
        print('\n')
        
        
finally:
    driver.quit()

컴퓨터 심화:


,0,1,2,3,4,5,6,7,8
0,입학년도,입학년도,입학년도,입학년도,2015~2016학년도,2015~2016학년도,2017~2022학년도,2023학년도,비고
1,구분,구분,구분,구분,2015~2016학년도,2015~2016학년도,2017~2022학년도,2023학년도,비고
2,교양,공통 교양,공통 교양,공통 교양,14~16학점,14~16학점,14~16학점,17학점,
3,교양,지정,기본소양,기본소양,9학점,9학점,9학점,9학점,
4,교양,지정,BSM,BSM,28학점,28학점,21학점,21학점,전산학영역불인정
5,교양,지정,BSM,수학,"미적분학및연습1, 확률및통계학","미적분학및연습1, 확률및통계학","미적분학및연습1, 확률및통계학","미적분학및연습1, 확률및통계학",필수
6,교양,지정,BSM,수학,이산수학,이산수학,공학선형대수학,"공학선형대수학, 이산수학",필수
7,교양,지정,BSM,과학,과학영역 ‘실험’교과목 1개 과목 선택 이수,과학영역 ‘실험’교과목 1개 과목 선택 이수,과학영역 ‘실험’교과목 1개 과목 선택 이수,과학영역 ‘실험’교과목 1개 과목 선택 이수,선택필수
8,전공,"단일 전공 이수자·다중 전공 이수자(주전공,복수전공)","단일 전공 이수자·다중 전공 이수자(주전공,복수전공)","단일 전공 이수자·다중 전공 이수자(주전공,복수전공)",84학점 (설계 12학점 포함),84학점 (설계 12학점 포함),84학점 (설계 12학점 포함),84학점 (설계 12학점 포함),전공전문교과목 50% 이상 이수
9,전공,"단일 전공 이수자·다중 전공 이수자(주전공,복수전공)","단일 전공 이수자·다중 전공 이수자(주전공,복수전공)","단일 전공 이수자·다중 전공 이수자(주전공,복수전공)","어드벤처디자인, 컴퓨터공학종합설계1, 컴퓨터공학종합설계2, 자료구조와실습, 개별연구...","어드벤처디자인, 컴퓨터공학종합설계1, 컴퓨터공학종합설계2, 자료구조와실습, 개별연구...","어드벤처디자인, 컴퓨터공학종합설계1, 컴퓨터공학종합설계2, 자료구조와실습, 개별연구...","어드벤처디자인, 컴퓨터공학종합설계1, 컴퓨터공학종합설계2, 자료구조와실습, 개별연구...",필수과목




컴퓨터 일반:


,0,1,2,3,4,5,6,7,8,9,10
0,입학년도,입학년도,입학년도,입학년도,입학년도,2015~2016학년도,2017~2019학년도,2020~2022학년도,2020~2022학년도,2023학년도,비고
1,구분,구분,구분,구분,구분,2015~2016학년도,2017~2019학년도,2020~2022학년도,2020~2022학년도,2023학년도,비고
2,교영,공통교양,공통교양,공통교양,공통교양,14~16학점,14~16학점,14~16학점,14~16학점,17학점,
3,교영,지정,기본소양,기본소양,기본소양,9학점,9학점,9학점,9학점,9학점,
4,교영,지정,BSM,BSM,BSM,28학점,21학점,21학점,21학점,21학점,전산학영역 불인정
5,교영,지정,BSM,BSM,수학,"미적분학및연습1, 확률및통계학","미적분학및연습1, 확률및통계학","미적분학및연습1, 확률및통계학","미적분학및연습1, 확률및통계학","미적분학및연습1, 확률및통계학",필수
6,교영,지정,BSM,BSM,수학,이산수학,공학선형대수학,공학선형대수학,공학선형대수학,"공학선형대수학, 이산수학",필수
7,교영,지정,BSM,BSM,과학,과학영역 ‘실험’교과목 1과목 선택 필수,과학영역 ‘실험’교과목 1과목 선택 필수,과학영역 ‘실험’교과목 1과목 선택 필수,과학영역 ‘실험’교과목 1과목 선택 필수,과학영역 ‘실험’교과목 1과목 선택 필수,선택 필수
8,전공,단일 전공 이수자,단일 전공 이수자,단일 전공 이수자,단일 전공 이수자,72학점,72학점,72학점,72학점,72학점,2021년 2월 졸업생 부터 일괄 적용
9,전공,다중 전공 이수자,다중 전공 이수자,주전공,주전공,45학점,45학점,45학점,45학점,45학점,2021년 2월 졸업생 부터 일괄 적용




멀티미디어:


...

,0,1,2,3
0,입학년도,입학년도,입학년도,2022 ~ 2023학년도
1,구분,구분,구분,2022 ~ 2023학년도
2,교양,공통교양,공통교양,17
3,교양,학문기초(자연과학영역),학문기초(자연과학영역),22 (아래 권장과목 참조)
4,전공,단일전공 이수자,단일전공 이수자,60
5,전공,복수전공 이수자,주전공,36
6,전공,복수전공 이수자,복수전공,36
7,졸업요건,취득학점,취득학점,130
8,졸업요건,평점평균,평점평균,2
9,졸업요건,외국어 시험 (TOEIC),외국어 시험 (TOEIC),700




데이터사이언스:


,0,1,2,3
0,입학년도,입학년도,입학년도,2022 ~ 2023학년도
1,구분,구분,구분,2022 ~ 2023학년도
2,교양,공통교양,공통교양,17
3,교양,학문기초(자연과학영역),학문기초(자연과학영역),18 (아래 권장과목 참조)
4,전공,단일전공 이수자,단일전공 이수자,60
5,전공,복수전공 이수자,주전공,36
6,전공,복수전공 이수자,복수전공,36
7,졸업 요건,취득학점,취득학점,130
8,졸업 요건,평점평균,평점평균,2
9,졸업 요건,외국어 시험 (TOEIC),외국어 시험 (TOEIC),700




엔터테인먼트:


,0,1,2,3
0,입학년도,입학년도,입학년도,2022 ~ 2023학년도
1,구분,구분,구분,2022 ~ 2023학년도
2,교양,공통교양,공통교양,17
3,교양,학문기초(자연과학영역),학문기초(자연과학영역),18 (아래 권장과목 참조)
4,전공,단일전공 이수자,단일전공 이수자,60
5,전공,복수전공 이수자,주전공,36
6,전공,복수전공 이수자,복수전공,36
7,졸업 요건,취득학점,취득학점,130
8,졸업 요건,평점평균,평점평균,2
9,졸업 요건,외국어 시험 (TOEIC),외국어 시험 (TOEIC),700
